## Importa as Bibliotecas

- **Requests** para conectar ao Open Street Map.
- **pprint** para manuseio dos dados.

In [1]:
import requests
from pprint import pprint

In [2]:
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

- Solicita o Código Postal e transfere para URL.

In [3]:
# postcode = 'G42 9AY'
postcode = str(input('Código postal: ')).strip()
response = requests.get(f"{BASE_URL}&postalcode={postcode}")

Transforma a resposta no formato json, se falhar, retorna o conteúdo da resposta.

In [4]:
try:
    data = response.json()
    print(data)
except:
    print("Error from server: " + str(response.content))

Error from server: b'<html>\n<head>\n<title>Access blocked</title>\n</head>\n<body>\n<h1>Access blocked</h1>\n\n<p>You have been blocked because you have violated the\n<a href="https://operations.osmfoundation.org/policies/nominatim/">usage policy</a>\nof OSM\'s Nominatim geocoding service. Please be aware that OSM\'s resources are\nlimited and shared between many users. The usage policy is there to ensure that\nthe service remains usable for everybody.</p>\n\n<p>Please review the terms and make sure that your\nsoftware adheres to the terms. You should in particular verify that you have set a\n<b>custom HTTP referrer or HTTP user agent</b> that identifies your application, and\nthat you are not overusing the service with massive bulk requests.</p>\n\n<p>If you feel that this block is unjustified or remains after you have adopted\nyour usage, you may contact the Nominatim system administrator at\nnominatim@openstreetmap.org to have this block lifted.</p>\n</body>\n</head>\n'


A partir da resposta em Json, é extraído a latitude e a longitude da localização informada.

In [5]:
latitude = data[0].get('lat')
longitude = data[0].get('lon')

(latitude, longitude)

NameError: name 'data' is not defined

Zipcodes:

In [6]:
zipcode = '01000-000'
response = requests.get(f'{BASE_URL}&postalcode={zipcode}')
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Bibliotecas

- **Folium:** Para mostrar o mapa

### Código

- Localização recebe latitude e longitude;
- a variável m recebe a localização no mapa;
- um marcador é adicionado na localização informada.

In [ ]:
import folium

location = float(latitude), float(longitude)

m = folium.Map(location=location, width=800, height=400)
folium.Marker(location, popup='Sua localização!').add_to(m)

m

